# FFNet metadata scraper
This notebook scrapes metadata from the [Fanfiction.net](https://www.fanfiction.net/). To make it work, put the URL for a particular fandom page (everything up to `&p=`) in as the *ScraperStem* value below, and set the range to be (1,some-number), where some-number is the final page of the paginated results for that fandom.

In [ ]:
#Install selenium for Python-based browser control
import sys
!{sys.executable} -m pip install selenium

In [ ]:
#Install undetected-chromedriver so you won't be identified as a bot
import sys
!{sys.executable} -m pip install undetected-chromedriver

In [ ]:
#You may need to run something on the Apple Terminal to make chromedriver work
#xattr -d com.apple.quarantine /usr/local/bin/chromedriver
#is what worked for me

In [1]:
#Function for removing HTML tags
def remove_html(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [2]:
#Import libraries
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from random import randint
import time
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
import re

In [3]:
#Create an empty Pandas datframe with columns for the metadata we're going to capture
ffdata = pd.DataFrame(columns=['Title', 'Ficlink', 'Author', 'AuthLink', 'Rating', 'Chapters', 'Words', 'Reviews', 'Favs', 'Follows', 'Updated', 'Published', 'Language', 'Genre', 'Characters', 'Blurb'])

The cell below is where you should put in the starting page for the fandom you want to scrape.

In [12]:
#List of all the languages on FFNet
langs = [' Afrikaans ', ' Bahasa Indonesia ', ' Bahasa Melayu ', ' Català ', ' Dansk ', ' Deutsch ', ' Eesti ', ' English ', ' Español ', ' Esperanto ', ' Filipino ', ' Français ', ' Hrvatski jezik ', ' Italiano ', ' Język polski ', ' LINGUA LATINA ', ' Magyar ', ' Nederlands ', ' Norsk ', ' Português ', ' Română ', ' Shqip ', ' Slovenčina ', ' Suomi ', ' Svenska ', ' Tiếng Việt ', ' Türkçe ', ' Íslenska ', ' čeština ', ' Ελληνικά ', ' България ', ' Русский ', ' Українська ', ' српски ', ' עברית ', ' العربية ', ' فارسی ', ' देवनागरी ', ' हिंदी ', ' ภาษาไทย ', ' 中文 ', ' 日本語 ', ' 한국어 ']
#List of all the genres on FFNet
genres = [' Adventure', ' Angst', ' Crime', ' Drama', ' Family', ' Fantasy', ' Friendship', ' General', ' Horror', ' Humor', ' Hurt/Comfort', ' Mystery', ' Parody', ' Poetry', ' Romance', ' Sci-Fi', ' Spiritual', ' Supernatural', ' Suspense', ' Tragedy', ' Western']

#Instantiate Selenium driver using the Undetected version
driver = uc.Chrome()

#This is the base URL for the fandom, up until p= (which indicates pagination)
ScraperStem = "https://www.fanfiction.net/book/Harry-Potter/?&srt=1&r=103&p="

The cell below is where you should define the page range, between 1 and whatever the largest page number is for that fandom.

In [22]:
#For each page in a particular range
#(We know the total range by looking at the first page for the fandom, and seeing what # the last page of results is)
for i in range(23827,25261):
    #Define authurl, authname, and ficlang as blank, to avoid errors if there's a problem with any of those values
    authurl = ''
    authname = ''
    ficlang = ''
    #URL is the stem + the current page number
    ScraperURL = ScraperStem + str(i)
    #Print what page number it is
    print(i)
    #Open the page
    driver.get(ScraperURL);
    #Get the source of the page
    pageSource = driver.page_source
    #Process the page source using Beautiful Soup
    soup = BeautifulSoup(pageSource)
    #Find every container div that includes a fic
    fics = soup.find_all('div', {'class': 'z-list'})
    #For each fic
    for fic in fics:
        #Find a URL with the class 'stitle'
        url = fic.find('a', {'class': 'stitle'})
        #Get the href attribut of this link, it's the fic URL
        ficurl = url.get('href')
        #Get the text of that link, it's the title
        fictitle = url.text
        #Find all the links within the fic container
        ficlinks = fic.find_all('a')
        #If there's 4 or more links within the fic container (Python starts counting with 0)...
        if len(ficlinks) >= 3:
            #The info for the author is probably the 3rd
            authhtml = ficlinks[2]
            #If authhtml exists...
            if authhtml is not None:
                #If authtml doesn't include the word 'reviews':
                if 'reviews' not in authhtml:
                    #The URL for the author profile is the link that starts with '/u/'
                    authurl = re.search(r'/u/([0-9]*)', str(authhtml)).group(0)
                    #The author name is the text inside the link for the author profile
                    authname = re.search(r'>(.*)<', str(authhtml)).group(1)
        #Otherwise...
        else:
            #If there's 3 or more links within the fic container...
            if len(ficlinks) >= 2:
                #The other possible author link could be the second one
                authhtmlalt = ficlinks[1]
                #Process like above
                if re.search(r'/u/([0-9]*)', str(authhtmlalt)) is not None:
                    authurl = re.search(r'/u/([0-9]*)', str(authhtmlalt)).group(0)
                    authname = re.search(r'>(.*)<', str(authhtmlalt)).group(1)
        #If the code can't find an author URL
        if authurl is None:
            #Set that to nothing
            authurl = ''
        #Same with author name
        if authname is None:
            authname = ''
        #If there is a link with the class 'reviews'
        if fic.find('a', {'class': 'reviews'}) is not None:
            #Define reviews as that value
            reviews = 'reviews'
        #Otherwise...
        else:
            #Set reviews to empty
            reviews=''
        #Find box with the blurb
        blurbbox = fic.find('div', {'class': 'z-indent'}).text
        #Search for the thing before the rating, which should be the blurb
        blurb = re.search(r'(.*)Rated:', blurbbox).group(1)
        #Find the box with the metadata
        metadata = fic.find('div', {'class': 'z-padtop2'})
        #Turn the metadata into a string
        metadatastring = str(metadata)
        #Strip the HTML
        metadatastring = remove_html(metadatastring)
        #Split it at the hyphen that separates values and turn it into a list
        metalist = metadatastring.split('-')
        #Define all the metadata as blank in case any is missing
        ficchars = ''
        favs = ''
        reviews = ''
        updated = ''
        follows = ''
        #For each item in the metadata list...
        for item in metalist:
            #If has the text 'Rated', it's a rating
            if 'Rated:' in item:
                rating = re.search(r'Rated: (.*)', item).group(1)
            #If it has the text 'Chapters', it's the chapter number
            elif 'Chapters:' in item:
                chapters = re.search(r'Chapters: (.*)', item).group(1)
            #If it has the text 'Words', it's the word count
            elif 'Words:' in item:
                words = re.search(r'Words: (.*)', item).group(1)
            #If it has the text 'Reviews', it's the review count
            elif 'Reviews:' in item:
                reviews = re.search(r'Reviews: (.*)', item).group(1)
            #If it has the text 'Favs', it's the favs count
            elif 'Favs:' in item:
                favs = re.search(r'Favs: (.*)', item).group(1)
            #If it has the text 'Follows', it's the follows count
            elif 'Follows:' in item:
                follows = re.search(r'Follows: (.*)', item).group(1)
            #If it has the text 'Updated', it's the updated date
            elif 'Updated:' in item:
                updated = re.search(r'Updated: (.*)', item).group(1)
            #If it has the text 'Published', it's the published date
            elif 'Published:' in item:
                published = re.search(r'Published: (.*)', item).group(1)
            #If it starts with something from the languages list, it's the language
            elif item.startswith(tuple(langs)) is not False:
                ficlang = item
            #If it starts with something from the genres list, it's the genres
            elif item.startswith(tuple(genres)) is not False:
                ficgenre = item
            #Otherwise if it starts with a capitalized word and then has another capital letter with a period...
            else:
                if re.search(r'[A-Z][a-z]* [A-Z]\.', item) is not None:
                    #It's the characters
                    ficchars = item
        #Create a new item with the data you've scraped
        newitem = {'Title': fictitle, 'Ficlink': ficurl, 'Author': authname, 'AuthLink': authurl, 'Rating': rating, 'Chapters': chapters, 'Words': words, 'Reviews': reviews, 'Favs': favs, 'Follows': follows, 'Updated': updated, 'Published': published, 'Language': ficlang, 'Genre': ficgenre, 'Characters': ficchars, 'Blurb': blurb}
        #Append the new item to the Pandas dataframe
        ffdata = ffdata.append(newitem, ignore_index=True)
    #Sleep 3-5 seconds before going to the next page
    sleep(randint(3,5))

23827
/u/243710
/u/243710
/u/319099
/u/222084
/u/44471
/u/50928
/u/50928
/u/350279
/u/354338
/u/354338
/u/354338
/u/354338
/u/354338
/u/351697
/u/252516
/u/252516
/u/250154
/u/250154
/u/250154
/u/250154
/u/250154
/u/250154
/u/250154
/u/50851
/u/50851
23828

/u/338948
/u/338948
/u/338948
/u/357904
/u/289256
/u/350331
/u/350331
/u/18136
/u/229458
/u/229458
/u/229458
/u/323345
/u/323345
/u/210009
/u/210009
/u/342947
/u/193732
/u/193732
/u/308877
/u/207980
/u/357959
/u/272166
/u/272166
/u/272166
23829
/u/336476
/u/163311
/u/163311
/u/311293
/u/215249
/u/215249
/u/215249
/u/303491
/u/303491
/u/87058
/u/332362
/u/282630
/u/172985
/u/187578
/u/187578
/u/187578
/u/169936
/u/169936
/u/326379
/u/326146
/u/326146
/u/274359
/u/352573
/u/174384
/u/174384
23830
/u/136277
/u/358626
/u/20550
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/227220
/u/359313
/u/354535
/u/359585
/u/341413
/u/341413
/u/353949
/u/328531
/u/328531
/u/328531
/u/328531
/u/

/u/117291
/u/117291
/u/117291
/u/279243
/u/194292
/u/194292
/u/194292
/u/194292
/u/194292
23861
/u/352198
/u/352198
/u/352198
/u/352198
/u/352198
/u/297258
/u/302408
/u/326307
/u/326307
/u/326307
/u/332805
/u/332805
/u/332805
/u/332805
/u/332805
/u/332805
/u/332805
/u/352995
/u/186396
/u/186396
/u/186446
/u/186396
/u/186396
/u/186396
/u/186396
23862
/u/335138
/u/335138
/u/289478
/u/289478
/u/289478
/u/272649
/u/290407
/u/290407
/u/290407
/u/290407
/u/290407
/u/221507
/u/221507
/u/221507
/u/311907
/u/167104
/u/167104
/u/38316
/u/307894
/u/307894
/u/339632
/u/292439
/u/182442
/u/295191
/u/295191
23863

/u/339195
/u/339195
/u/339195
/u/339195
/u/339195
/u/339195
/u/328961
/u/328961
/u/328961
/u/132044
/u/132044
/u/132044
/u/30072
/u/50928
/u/50928
/u/50928
/u/50928
/u/50928
/u/50928
/u/354307
/u/251422
/u/351694
/u/55168
/u/55168
23864

/u/211584
/u/7826
/u/338509
/u/289110
/u/289110
/u/289110
/u/289110
/u/289110
/u/353396
/u/353396
/u/353396
/u/353396
/u/233844
/u/233844
/u/295816
/u/318

/u/231457
/u/198834
/u/198834
/u/115049
/u/329778
/u/322838
/u/234646
/u/234646
/u/292573
/u/292573
/u/292573
/u/292573
/u/347904
/u/225815
/u/225815
/u/225815
/u/225815
23895

/u/340838
/u/348730
/u/340838
/u/340838
/u/234551
/u/234551
/u/234551
/u/186226
/u/316485
/u/348786
/u/295816
/u/348631
/u/348631
/u/196640
/u/196640
/u/318932
/u/347802
/u/347802
/u/347802
/u/347802
/u/324021
/u/324021
/u/324021
/u/347048
23896
/u/302050
/u/348636
/u/348636
/u/348636
/u/348636
/u/24466
/u/24466
/u/24466
/u/328785
/u/328785
/u/145185
/u/145185
/u/145185
/u/238140
/u/311219
/u/311219
/u/311219
/u/273781
/u/281795
/u/281795
/u/281795
/u/281795
/u/281795
/u/281795
/u/281795
23897

/u/250030
/u/252245
/u/211936
/u/243013
/u/243013
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/54165
/u/223768
/u/39379
/u/272799
/u/272799
/u/254642
/u/71047
/u/71047
/u/71047
23898


/u/247739
/u/247739
/u/247739
/u/247739
/u/249262
/u/249262
/u/342945
/u/342945
/u/80067
/

/u/260926
23928
/u/275521
/u/280115
/u/280115
/u/309087
/u/100447
/u/257454
/u/257454
/u/163742
/u/260306
/u/322872
/u/322872
/u/322872
/u/334902
/u/334902
/u/334902
/u/334902
/u/334902
/u/332827
/u/332827
/u/332827
/u/311900
/u/311900
/u/311900
/u/240577
/u/38855
23929



/u/239810
/u/306290
/u/343296
/u/306290
/u/314238
/u/235862
/u/256457
/u/296566
/u/296566
/u/296566
/u/296566
/u/285967
/u/285967
/u/285967
/u/285967
/u/285967
/u/285967
/u/338063
/u/318448
/u/311439
/u/311439
/u/229979
23930

/u/275727
/u/332799
/u/332799
/u/332799
/u/332799
/u/332799
/u/332799
/u/332799
/u/319237
/u/256165
/u/256165
/u/256165
/u/256165
/u/292069
/u/292069
/u/292069
/u/292069
/u/292069
/u/292069
/u/292069
/u/341260
/u/340332
/u/340332
/u/308877
23931
/u/219928
/u/342915
/u/342915
/u/280782
/u/280782
/u/280782
/u/280782
/u/280782
/u/277766
/u/277766
/u/277766
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
/u/25235
23932
/u/312441
/

/u/307637
/u/307637
/u/307637
/u/279989
/u/172822
/u/154336
/u/221754
/u/100219
/u/201959
/u/201959
/u/201959
/u/201959
/u/242247
/u/304777
/u/304777
/u/304777
/u/304777
/u/319013
/u/319013
/u/81622
/u/211621
23963
/u/315834
/u/324181
/u/211175
/u/211175
/u/254945
/u/254945
/u/254945
/u/254945
/u/254945
/u/154075
/u/154075
/u/154075
/u/329770
/u/329770
/u/325243
/u/266043
/u/336572
/u/336572
/u/39291
/u/39291
/u/307239
/u/307239
/u/307239
/u/324398
/u/324398
23964

/u/210288
/u/210288
/u/310111
/u/311219
/u/319963
/u/319963
/u/302719
/u/302719
/u/302719
/u/302719
/u/302719
/u/302719
/u/302719
/u/197192
/u/197192
/u/197192
/u/197192
/u/250947
/u/253800
/u/285335
/u/141567
/u/141567
/u/141567
/u/334032
23965




/u/209558
/u/42080
/u/169781
/u/169781
/u/306357
/u/258967
/u/336214
/u/336214
/u/336214
/u/336214
/u/318754
/u/201435
/u/201435
/u/321015
/u/333757
/u/299679
/u/299679
/u/299679
/u/124165
/u/124165
/u/124165
23966

/u/96376
/u/38985
/u/224879
/u/96376
/u/96376
/u/96376
/u/96376


...

/u/4627
/u/5739
/u/5739
/u/30302
/u/19072
/u/19072
/u/19072
25037







/u/48808
/u/53503
/u/55556
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/19705
/u/14489
/u/14489
/u/14489
/u/14489
/u/14489
25038
/u/22350
/u/51460
/u/51460
/u/51460
/u/51460
/u/51460
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/15326
/u/33165
/u/43709
/u/24899
/u/31156
/u/31156
/u/15326
/u/54474
25039












/u/43841
/u/5454
/u/52494
/u/52990
/u/33814
/u/33814
/u/33814
/u/33814
/u/33814
/u/33814
/u/33814
/u/33814
/u/33814
25040




















/u/40942
/u/36579
/u/53997
/u/23255
/u/23255
25041



/u/40963
/u/50093
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/26953
/u/51191
/u/17561
/u/20121
/u/43025
/u/17164
/u/34902
25042

/u/35225
/u/52600
/u/52600
/u/52600
/u/52600
/u/52600
/u/52600
/u/52600
/u/52600
/u/52600
/u/25140
/u/52932
/u/3513

/u/8036
25079
/u/26992
/u/15943
/u/17404
/u/16733
/u/8036
/u/8036
/u/6965
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
/u/16733
25080





/u/37912
/u/41343
/u/41343
/u/33238
/u/33238
/u/33238
/u/21550
/u/21550
/u/35314
/u/9485
/u/9485
/u/18513
/u/18513
/u/18513
/u/18513
/u/18513
/u/18513
/u/18513
/u/18513
/u/18513
25081

/u/38778
/u/41411
/u/42165
/u/42165
/u/42165
/u/42165
/u/42318
/u/42318
/u/42318
/u/42318
/u/30096
/u/15334
/u/15334
/u/15334
/u/15334
/u/15334
/u/26674
/u/23877
/u/22773
/u/22773
/u/9559
/u/41931
/u/41931
/u/32951
25082






/u/42165
/u/42165
/u/42165
/u/42165
/u/42165
/u/42165
/u/31829
/u/31829
/u/31829
/u/21550
/u/21550
/u/21550
/u/15771
/u/10255
/u/16597
/u/13869
/u/21687
/u/7213
/u/7599
25083
/u/3677
/u/14491
/u/14491
/u/20734
/u/38265
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23199
/u/23

/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
/u/34148
25126


















/u/34187
/u/34187
/u/34187
/u/34187
/u/34255
/u/34255
/u/34255
25127
/u/32246
/u/32246
/u/32246
/u/33264
/u/33264
/u/33264
/u/33264
/u/33264
/u/33264
/u/33264
/u/33264
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
/u/27427
25128














/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/31733
/u/33772
25129
/u/18632
/u/18632
/u/18632
/u/18632
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/30988
/u/32853
/u/32853
/u/32853
/u/32853
/u/32853
/u/32853
/u/32241
25130












/u/24454
/u/24454
/u/24454
/u/24454
/u/24454
/u/24454
/u/24454
/u/6528
/u/6528
/u/6528
/u/6528
/u/6528
/u/6528
25131

/u/33300
/u/33300
/u/33300
/u/33300
/u/33300
/u/33300
/u/33300
/u/32028
/u/3202

/u/25349
/u/25353
/u/25353
/u/25353
/u/25353
/u/25353
/u/25353
25174

/u/23235
/u/23235
/u/24933
/u/24933
/u/24933
/u/24933
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
/u/9039
25175










/u/20362
/u/20362
/u/20362
/u/20362
/u/20362
/u/20362
/u/24940
/u/24952
/u/24952
/u/19066
/u/19066
/u/21594
/u/21594
/u/21594
/u/21594
25176





/u/24851
/u/24851
/u/24851
/u/7732
/u/7732
/u/7732
/u/24578
/u/24578
/u/24578
/u/24544
/u/24544
/u/19066
/u/19066
/u/19066
/u/19066
/u/19066
/u/19066
/u/19066
/u/19066
/u/15613
25177
/u/21267
/u/21267
/u/21267
/u/21267
/u/21267
/u/21267
/u/24512
/u/24512
/u/24512
/u/24512
/u/24512
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/24364
/u/15411
/u/15411
25178


/u/18786
/u/18786
/u/18786
/u/18786
/u/18786
/u/18786
/u/18786
/u/18786
/u/18786
/u/24389
/u/18128
/u/14200
/u/24415
/u/24415
/u/24415
/u/24415
/u/24415
/

/u/17020
/u/17020
/u/17020
/u/17020
/u/17020
/u/17020
/u/17020
/u/11230
/u/11230
/u/16545
25219


















/u/11623
/u/11623
/u/11623
/u/15871
/u/15871
/u/15871
/u/15871
25220
/u/13029
/u/13029
/u/13029
/u/13029
/u/13029
/u/13029
/u/13029
/u/13029
/u/16691
/u/16691
/u/16290
/u/14845
/u/14845
/u/14845
/u/14845
/u/14845
/u/14845
/u/16555
/u/16555
/u/16555
/u/16555
/u/16555
/u/16497
/u/16497
/u/16497
25221
























/u/16265
25222

/u/7220
/u/7220
/u/15556
/u/15556
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16354
/u/16278
/u/16278
/u/16278
/u/16278
/u/16278
/u/16278
/u/16278
25223




/u/15360
/u/15466
/u/15501
/u/15501
/u/15501
/u/16024
/u/16024
/u/16024
/u/16024
/u/16024
/u/16024
/u/16037
/u/16037
/u/16077
/u/16077
/u/16077
/u/16022
/u/16022
/u/16008
/u/16008
/u/16008
25224










/u/15783
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/11453
/u/15699


In [19]:
#Display the data
ffdata

,Title,Ficlink,Author,AuthLink,Rating,Chapters,Words,Reviews,Favs,Follows,Updated,Published,Language,Genre,Characters,Blurb
0,Lily und James a true Lovestory,/s/2934287/1/Lily-und-James-a-true-Lovestory,,,K,1,"1,280",6,,1,,"May 11, 2006",,Romance/Humor,"Lily Evans P., James P.",Wir das heißt Lily Witchcraft und ich haben un...
1,Marauders Mischief,/s/2906995/1/Marauders-Mischief,Aka Abby,/u/984310,T,12,"10,149",9,7,5,"May 11, 2006","Apr 22, 2006",English,Romance/Adventure,"James P., Lily Evans P.",".::Love, adventure, and anything else that cou..."
2,Fate is Sealed,/s/2934251/1/Fate-is-Sealed,Aka Abby,/u/984310,K,1,609,14,5,3,,"May 11, 2006",English,Romance/Poetry,"Hermione G., Severus S.","Hermione and Severus in a Potions lesson, both..."
3,T'oublieras,/s/2934232/1/T-oublieras,Aka Abby,/u/984310,K+,1,"1,480",6,2,1,,"May 11, 2006",English,Romance/Tragedy,"Hermione G., Draco M.",Estce qu'Hermione réussira à survivre à sa rup...
4,Paranoïa Sucrée,/s/2920847/1/Parano%C3%AFa-Sucr%C3%A9e,yochu,/u/784825,K,2,"8,088",58,39,9,"May 11, 2006","May 3, 2006",English,Humor/Romance,"Harry P., Draco M.","¤TwoShot¤ La paranoïa, c’est bien. Le glucose,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121365,Rosemary and Thyme,/s/1281324/1/Rosemary-and-Thyme,SkyAngeli Akari,/u/186674,T,1,"3,260",53,87,11,,"Mar 24, 2003",English,Romance,"Severus S., Hermione G.",Hermione's friends talk her into undergoing pa...
121366,Mistaken Identity,/s/1281314/1/Mistaken-Identity,SkyAngeli Akari,/u/186674,T,1,"1,316",26,14,,,"Mar 24, 2003",English,Humor/Romance,"Harry P., Ginny W.",A one-shot. A conversation between friends cre...
121367,Harry Potter and the Book of Merlin,/s/710026/1/Harry-Potter-and-the-Book-of-Merlin,gotta B writin,/u/143394,T,7,"18,565",63,10,4,"Mar 24, 2003","Apr 8, 2002",English,Supernatural/Adventure,,*Ch 6 Uploaded* Harry's summer is cut short wh...
121368,Harry Potter and ummm ME! discontinued,/s/1272529/1/Harry-Potter-and-ummm-ME-disconti...,Dreamer of Fantasy,/u/243710,K+,2,"2,678",12,,,"Mar 24, 2003","Mar 17, 2003",English,Humor/Humor,,"I will not lie to you. This is chaotic, confus..."


In [23]:
#Strip any tabs or newlines
ffdataclean = ffdata.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=False)
ffdataclean

,Title,Ficlink,Author,AuthLink,Rating,Chapters,Words,Reviews,Favs,Follows,Updated,Published,Language,Genre,Characters,Blurb
0,Lily und James a true Lovestory,/s/2934287/1/Lily-und-James-a-true-Lovestory,,,K,1,"1,280",6,,1,,"May 11, 2006",,Romance/Humor,"Lily Evans P., James P.",Wir das heißt Lily Witchcraft und ich haben un...
1,Marauders Mischief,/s/2906995/1/Marauders-Mischief,Aka Abby,/u/984310,T,12,"10,149",9,7,5,"May 11, 2006","Apr 22, 2006",English,Romance/Adventure,"James P., Lily Evans P.",".::Love, adventure, and anything else that cou..."
2,Fate is Sealed,/s/2934251/1/Fate-is-Sealed,Aka Abby,/u/984310,K,1,609,14,5,3,,"May 11, 2006",English,Romance/Poetry,"Hermione G., Severus S.","Hermione and Severus in a Potions lesson, both..."
3,T'oublieras,/s/2934232/1/T-oublieras,Aka Abby,/u/984310,K+,1,"1,480",6,2,1,,"May 11, 2006",English,Romance/Tragedy,"Hermione G., Draco M.",Estce qu'Hermione réussira à survivre à sa rup...
4,Paranoïa Sucrée,/s/2920847/1/Parano%C3%AFa-Sucr%C3%A9e,yochu,/u/784825,K,2,"8,088",58,39,9,"May 11, 2006","May 3, 2006",English,Humor/Romance,"Harry P., Draco M.","¤TwoShot¤ La paranoïa, c’est bien. Le glucose,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157215,A Candle in the WInd,/s/9163/1/A-Candle-in-the-WInd,Mimi Jae,/u/6216,K,1,457,72,1,,,"Jan 23, 2000",English,Drama,,My first songfic and I picked a song that I kn...
157216,The Race,/s/4712/1/The-Race,Mimi Jae,/u/6216,K+,1,"2,035",117,11,6,,"Jan 22, 2000",English,Romance,,James said he'd do anything to win Lily's hear...
157217,THE HORRORS OF AZKABAN,/s/8882/1/THE-HORRORS-OF-AZKABAN,Mimi Jae,/u/6216,K+,1,240,17,,,,"Jan 21, 2000",English,Poetry,,A poem describing the terrors and horrors of A...
157218,HARRY POTTER IN THE CARE OF DUMBLEDORE,/s/8881/1/HARRY-POTTER-IN-THE-CARE-OF-DUMBLEDORE,Mimi Jae,/u/6216,K,1,209,31,3,,,"Jan 21, 2000",English,Poetry,,A nice poem about Harry and Dumbledore.


In [24]:
#Write the cleaned data to a TSV file
ffdataclean.to_csv('/Users/qad/Documents/ffdata_last.tsv', index=False, sep='\t')